# Character-Level LSTM in PyTorch

Constructing a character-level LSTM with PyTorch:
The network will train character by character on some text, then generate new text character by character.Training is done on Anna Karenina. **This model will be able to generate new text based on the text from the book!**

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

Loading the required resources for data loading and model creation.

In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

## Load in Data

Loading the Anna Karenina text file and convert it into integers for the network to use. 

In [2]:
# open text file and read in data as `text`
with open('data/anna.txt', 'r') as f:
    text = f.read()

Check out the first 100 characters. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

### Tokenization

Creating a couple **dictionaries** to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [4]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

Observing same characters from above, encoded as integers.

In [5]:
encoded[:100]

array([23, 11,  4, 29, 26, 18, 14, 81, 80, 78, 78, 78, 27,  4, 29, 29, 28,
       81,  5,  4, 61, 34,  3, 34, 18, 54, 81,  4, 14, 18, 81,  4,  3,  3,
       81,  4,  3, 34, 47, 18,  6, 81, 18, 67, 18, 14, 28, 81,  7,  8, 11,
        4, 29, 29, 28, 81,  5,  4, 61, 34,  3, 28, 81, 34, 54, 81,  7,  8,
       11,  4, 29, 29, 28, 81, 34,  8, 81, 34, 26, 54, 81, 58, 36,  8, 78,
       36,  4, 28, 72, 78, 78, 60, 67, 18, 14, 28, 26, 11, 34,  8])

## Pre-processing the data

In the char-RNN image above, the LSTM expects an input that is **one-hot encoded** meaning that each character is converted into an integer (via the created dictionary) and *then* converted into a column vector where only it's corresponding integer index will have the value of 1 and the rest of the vector will be filled with 0's. 

In [6]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [7]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[ 0.  0.  0.  1.  0.  0.  0.  0.]
  [ 0.  0.  0.  0.  0.  1.  0.  0.]
  [ 0.  1.  0.  0.  0.  0.  0.  0.]]]


## Making training mini-batches


To train on this data, creating mini-batches for training are necessary. Batches are needed to be multiple sequences of some desired number of sequence steps. Considering a simple example, the batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>

In this example, the encoded characters  are taken (passed in as the `arr` parameter) and are splitted into multiple sequences, given by `batch_size`. Each of the sequences will be `seq_length` long.

### Creating Batches

**1. The first thing is, discarding some of the text so just completely full mini-batches will remain. **

Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences in a batch) and $M$ is the seq_length or number of time steps in a sequence. Then, to get the total number of batches, $K$, can be made from the array `arr`, by dividing the length of `arr` by the number of characters per batch. After knowing the number of batches, the total number of characters to keep from `arr`can be gotten, $N * M * K$.

**2. After that, it needs to split `arr` into $N$ batches. ** 

This can be done by using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. $N$ sequences in a batch is needed, so let's make that the size of the first dimension. For the second dimension, `-1` as a placeholder in the size can be used, it'll fill up the array with the appropriate data . After this, an array that is $N \times (M * K)$ can be seen.

**3. Now with this array, it is needed to iterate through it to get our mini-batches. **

The idea is each batch is a $N \times M$ window on the $N \times (M * K)$ array. For each subsequent batch, the window moves over by `seq_length`. Also both the input and target arrays should built. The targets are just the inputs shifted over by one character. The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of tokens in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `seq_length` wide.

>Now, Write the code for creating batches in the function below

In [8]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array that make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    ##  Get the number of batches 
    n_batches = len(arr)// (batch_size * seq_length)
    #number of characters in mini-batches
    mini_batch = batch_size * seq_length
    ##  Keep only enough characters to make full batches
    arr = arr[:n_batches* mini_batch]
    
    ##  Reshape into batch_size rows #make the num of rows equal to batch size
    arr = arr.reshape((batch_size , -1))
    
    ##  Iterate over the batches using a window of size seq_length
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, first it should be in the same size as x
        y = np.zeros_like(x)
        #shifted by one
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]

        yield x, y
       

### Test Your Implementation

 Using a batch size of 8 and 50 sequence steps.

In [9]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)


In [10]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[23 11  4 29 26 18 14 81 80 78]
 [54 58  8 81 26 11  4 26 81  4]
 [18  8 68 81 58 14 81  4 81  5]
 [54 81 26 11 18 81 64 11 34 18]
 [81 54  4 36 81 11 18 14 81 26]
 [64  7 54 54 34 58  8 81  4  8]
 [81 71  8  8  4 81 11  4 68 81]
 [33 16  3 58  8 54 47 28 72 81]]

y
 [[11  4 29 26 18 14 81 80 78 78]
 [58  8 81 26 11  4 26 81  4 26]
 [ 8 68 81 58 14 81  4 81  5 58]
 [81 26 11 18 81 64 11 34 18  5]
 [54  4 36 81 11 18 14 81 26 18]
 [ 7 54 54 34 58  8 81  4  8 68]
 [71  8  8  4 81 11  4 68 81 54]
 [16  3 58  8 54 47 28 72 81 39]]


---
## Defining the network with PyTorch

Define the network.

<img src="assets/charRNN.png" width=500px>

Using PyTorch to define the architecture of the network. For doing this, first the layers and operations need to be defined. Then, define a method for the forward pass. At the end, define a  method for predicting characters.

### Model Structure

In `__init__` the suggested structure is as follows:
* Create and store the necessary dictionaries
* Define an LSTM layer that takes as params: an input size (the number of characters), a hidden layer size `n_hidden`, a number of layers `n_layers`, a dropout probability `drop_prob`, and a batch_first boolean (True, because of batch)
* Define a dropout layer with `dropout_prob`
* Define a fully-connected layer with params: input size `n_hidden` and output size (the number of characters)
* Finally, initialize the weights 


---
### LSTM Inputs/Outputs

Create a basic [LSTM layer](https://pytorch.org/docs/stable/nn.html#lstm) as follows

```python
self.lstm = nn.LSTM(input_size, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
```

where `input_size` is the number of characters this cell expects to see as sequential input, and `n_hidden` is the number of units in the hidden layers in the cell. Then dropout needs to be added with a specified probability; this will automatically add dropout to the inputs or outputs. Finally, in the `forward` function, we can stack up the LSTM cells into layers using `.view`. With this, we pass in a list of cells and it will send the output of one cell into the next cell.

Also an initial hidden state of all zeros needs to be created:
```python
self.init_hidden()
```

In [11]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [12]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        # define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, dropout= drop_prob, batch_first = True)
        # define a drop out layer with drop_out probability
        self.dropout = nn.Dropout(drop_prob)
        #define a fully connected layer 
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
      
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        #Get the outputs and the new hidden state from the lstm
        output_lstm, hidden = self.lstm(x, hidden)
        #pass through dropout
        output = self.dropout(output_lstm)
        #stacking up the lstm cells into layer
        output = output.contiguous() .view(-1, self.n_hidden)
        #pass through fully connected layer
        out = self.fc(output)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

## Time to train

The train function gives this ability to set the number of epochs, the learning rate, and other parameters.

Below an Adam optimizer and cross entropy loss is used since character class scores as output is required. Calculating the loss and perform backpropagation is necessary.

A couple of details about training: 
>* Within the batch loop, we should detach the hidden state from its history; this time setting it equal to a new *tuple* variable because an LSTM has a hidden state that is a tuple of the hidden and cell states.
* We use [`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html) to help prevent exploding gradients.

In [13]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length))
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length))
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

## Instantiating the model

Before training,the network itself, with some given hyperparameters should be created. Then, define the mini-batches sizes, and start training.

In [14]:
# define the hyperparameter and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


### Set your training hyperparameters!

In [15]:
batch_size = 128
seq_length = 100
n_epochs =  20 # start small if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/20... Step: 10... Loss: 3.2509... Val Loss: 3.2151
Epoch: 1/20... Step: 20... Loss: 3.1460... Val Loss: 3.1400
Epoch: 1/20... Step: 30... Loss: 3.1406... Val Loss: 3.1252
Epoch: 1/20... Step: 40... Loss: 3.1081... Val Loss: 3.1182
Epoch: 1/20... Step: 50... Loss: 3.1411... Val Loss: 3.1163
Epoch: 1/20... Step: 60... Loss: 3.1151... Val Loss: 3.1135
Epoch: 1/20... Step: 70... Loss: 3.1037... Val Loss: 3.1094
Epoch: 1/20... Step: 80... Loss: 3.1101... Val Loss: 3.0999
Epoch: 1/20... Step: 90... Loss: 3.0972... Val Loss: 3.0796
Epoch: 1/20... Step: 100... Loss: 3.0374... Val Loss: 3.0285
Epoch: 1/20... Step: 110... Loss: 2.9394... Val Loss: 2.9154
Epoch: 1/20... Step: 120... Loss: 2.8328... Val Loss: 2.8293
Epoch: 1/20... Step: 130... Loss: 2.7868... Val Loss: 2.7140
Epoch: 2/20... Step: 140... Loss: 2.6772... Val Loss: 2.6306
Epoch: 2/20... Step: 150... Loss: 2.5924... Val Loss: 2.5464
Epoch: 2/20... Step: 160... Loss: 2.5412... Val Loss: 2.4878
Epoch: 2/20... Step: 170... Loss:

## Getting the best model

To set your hyperparameters to get the best performance, you'll want to watch the training and validation losses. If your training loss is much lower than the validation loss, you're overfitting. Increase regularization (more dropout) or use a smaller network. If the training and validation losses are close, you're underfitting so you can increase the size of the network.

## Hyperparameters

Here are the hyperparameters for the network.

In defining the model:
* `n_hidden` - The number of units in the hidden layers.
* `n_layers` - Number of hidden LSTM layers to use.

We assume that dropout probability and learning rate will be kept at the default, in this example.

And in training:
* `batch_size` - Number of sequences running through the network in one pass.
* `seq_length` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lr` - Learning rate for training

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `n_hidden` and `n_layers`. I would advise that you always use `n_layers` of either 2/3. The `n_hidden` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `n_hidden` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

## Checkpoint

After training, we'll save the model so we can load it again later if we need too. Here I'm saving the parameters needed to create the same architecture, the hidden layer hyperparameters and the text characters.

In [16]:
# change the name, for saving multiple files
model_name = 'rnn_x_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

---
## Making Predictions

Now that the model is trained, we'll want to sample from it and make predictions about next characters! To sample, we pass in a character and have the network predict the next character. Then we take that character, pass it back in, and get another predicted character. Just keep doing this and you'll generate a bunch of text!

### A note on the `predict`  function

The output of our RNN is from a fully-connected layer and it outputs a **distribution of next-character scores**.

> To actually get the next character, we apply a softmax function, which gives us a *probability* distribution that we can then sample to predict the next character.

### Top K sampling

Our predictions come from a categorical probability distribution over all the possible characters. We can make the sample text and make it more reasonable to handle (with less variables) by only considering some $K$ most probable characters. This will prevent the network from giving us completely absurd characters while allowing it to introduce some noise and randomness into the sampled text. Read more about [topk, here](https://pytorch.org/docs/stable/torch.html#torch.topk).


In [17]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

### Priming and generating text 

Typically you'll want to prime the network so you can build up a hidden state. Otherwise the network will start out generating characters at random. In general the first bunch of characters will be a little rough since it hasn't built up a long history of characters to predict from.

In [18]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [19]:
print(sample(net, 1000, prime='Anna', top_k=5))

Anna. And
went away to his wail, turned at his white small, and the
conversation was high straighe that she was saying he was not answered
in the senses, trathering at an old man as he would be done in a strange
for the path, had taken into a little best thought in her eyes, but, and
a long while she had not been thanked a country, he stood, and shit
her and taken the fiests and seried, and at his soul, and stopped, and
she felt at the same, that the stalling of the side-simple would
should be to be terrible for the chair, and and the study. But the
marshal of that matter, there was so the carciate and all the table
when he was not to come fat together and the party of her face. He
had not taken a forest of her people--the children were dissitting
the string of the contrary, a shade of such a stand townromed
the same tincor, they had suddenly her hand. But she was still
satisfied, but the sound of the countess and service, he had begun
to come only thinking about him, and all these wil

## Loading a checkpoint

In [20]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_x_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

In [21]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

And Levin said
to him, and though he was struggling. She saw that they thinking it
the more and to care, too say to him, his soficiar stacks. But she was
and started at the count he was this terror to be done into the
country and seeing her."

"Yes, I don't know;" he said in.

"You can't stay that you should have anyone. I should see her, that
it's a coming and meant and more, but I have an artastion of more
concertation. I would be, both in the same another were so much in sickly,
and the man, the forests is something so much when I do nothing but
the children, and I saw a merchant, and he's business, I won't go to the
peasant of myself..."

"We do nother, and went out to her, tender than myself."

At the contrary had been breaking off, he could not go into the
receivanion, which she would say those whole centary was trying to be
so stopping in the chief clear, and his heart something that was
not faticial than the forces that is all she had been settled that
to her weally.

"Yes, but